<a href="https://colab.research.google.com/github/amengemeda/ISproject-2/blob/main/Amharic_Hate_Speech_detection_using_fine_tuned_ROBERTA(Trainer_API).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#installing the transformers package
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.2 MB/s 
     |████████████████████████████████| 7.6 MB 50.3 MB/s 
     |████████████████████████████████| 182 kB 67.9 MB/s 


In [2]:
#installing the datasets package
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 3.5 MB/s 
     |████████████████████████████████| 212 kB 51.0 MB/s 
     |████████████████████████████████| 132 kB 53.4 MB/s 
     |████████████████████████████████| 127 kB 65.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
#importing the datasets package
from datasets import Dataset
import datasets
#import load metric for model evaluation
from datasets import load_metric

In [4]:
#import numpy and pandas for mathematical computation and data manipulation respectively 
import numpy as np
import pandas as pd
#import drive package to connect this colab file with the drive where the data will be retrived from
from google.colab import drive
#import the pipeline of transformers
from transformers import pipeline
#import AutoTokenizer for tokenization purposes
from transformers import AutoTokenizer


In [5]:
#import torch
import torch
#import Data loader from torch
from torch.utils.data import DataLoader
#import an optimizer
from torch.optim import AdamW
#import tqdm for a progress bar
from tqdm.auto import tqdm

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
#mount google drive to access the dataset directly from the drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [8]:
#fetch the dataset from the drive
Labels=pd.read_csv('/content/drive/MyDrive/Machine Learning/Data/Amharic-Hate-Speech-Dataset/Labels.txt',header=None)
Posts=pd.read_csv('/content/drive/MyDrive/Machine Learning/Data/Amharic-Hate-Speech-Dataset/Posts.txt',header=None)

In [9]:
#naming the columns
Labels.columns = ["labels"]
Posts.columns = ["post"]

In [10]:
#encoding the classes into numerical data
Labels = Labels.replace(['Free', 'Free ','Hate'],[0,0,1]) 

In [11]:
#check the encoded label data
Labels.head(10)

,labels
0,0
1,0
2,0
3,0
4,1
5,0
6,0
7,0
8,1
9,0


In [12]:
#check the Amharic data
Posts.head(1000)

,post
0,አስቀድሜ ጥያቄዬ በጨዋነት በውስጥ መስመር እንዲደርስዎ አድርጌ ፍትህን ለ...
1,እነዚህን ወሳኝ ጉዳዮችን የሚያስፈፅም አካል እንዲቋቋምና ክትትል እንዲደ...
2,የአማራ ህዝብ በአእምሮ ክንፉ ያልበረረበት ጥበብና ፍልስፍና ያልከፈተው የ...
3,ከአማራ ህዝብ የሀገሪቱ ዘርፈ ብዙ እውቀት መንጭቶ የሞላበትከሙላቱም በመል...
4,ዛሬ በየትኛውም መለኪያ ይሁን መመዘኛ ኢትዮጵያዊነት የሚንፀባረቀው በአማራ...
...,...
995,መፈንቅለመንግስት ከሽፏልእንዴ የየክልሉ ህዝብ ምን ይጠብቃል ሆ ብሎ ሄዶ ...
996,ሰው በሩን እንደፈለገ መክፈት እንደፈለገ መዝጋት ይችላል የምን አውቅልሻ...
997,ክክክክክ የሱዳን ዜጋ ናችሁ እንዴ ባለ ነጠላ ጎጃሜ ሁላ
998,የምን ማለቃቀስ ነው ምድረ ፎካሪ ሁላ


In [13]:
#merge the datasets
Frames = [Labels, Posts]
Merged = pd.concat(Frames, axis=1)


In [54]:
#preview of merged data
Merged

,labels,post
0,0,አስቀድሜ ጥያቄዬ በጨዋነት በውስጥ መስመር እንዲደርስዎ አድርጌ ፍትህን ለ...
1,0,እነዚህን ወሳኝ ጉዳዮችን የሚያስፈፅም አካል እንዲቋቋምና ክትትል እንዲደ...
2,0,የአማራ ህዝብ በአእምሮ ክንፉ ያልበረረበት ጥበብና ፍልስፍና ያልከፈተው የ...
3,0,ከአማራ ህዝብ የሀገሪቱ ዘርፈ ብዙ እውቀት መንጭቶ የሞላበትከሙላቱም በመል...
4,1,ዛሬ በየትኛውም መለኪያ ይሁን መመዘኛ ኢትዮጵያዊነት የሚንፀባረቀው በአማራ...
...,...,...
29995,1,በአሉ የሁሉም ኢትዮጵያዊ ስላልሆነ በኦሮምኛው ቢለፋደድ ምን አገባን
29996,0,ተባረክ አብቹ ፈር ቀዳጅ ስለሆንህ መጋረጃው መቀደድ ስለጀመረ
29997,0,እስከ አሁን አንተ ብቻ ነው በ መፅሀፍ ያልቻልከው አንተም ታሪክ እ...
29998,1,ህገወጥት ጠቅላይ ሚንስትር ፅቤት የተፈቀደ ሆኖ ህዝብን እንዴት ህግ አክ...


In [55]:
train_val_df, test_dataset = train_test_split(Merged, test_size=0.20, random_state=42)
train_dataset, evaluation_dataset = train_test_split(train_val_df, test_size=0.10, random_state=42)
print('Training dataset shape: ', train_dataset.shape)
print('Validation dataset shape: ', evaluation_dataset.shape)
print('Testing dataset shape: ', test_dataset.shape)

Training dataset shape:  (21600, 2)
Validation dataset shape:  (2400, 2)
Testing dataset shape:  (6000, 2)


In [56]:
#Divide the dataset into train and test categories 
msk = np.random.rand(len(Merged)) < 0.8
train_dataset = Merged[msk]
test_dataset = Merged[~msk]

In [57]:
#convert format of the dataset to HuggingFace Dataset from Pandas DataFrame
test_dataset=Dataset.from_pandas(test_dataset)

In [58]:
#convert the format of the dataset to HuggingFace Dataset from Pandas DataFrame
train_dataset=Dataset.from_pandas(train_dataset)

In [59]:
#convert the format of the dataset to HuggingFace Dataset from Pandas DataFrame
evaluation_dataset=Dataset.from_pandas(evaluation_dataset)

In [60]:
#preview of the dataset after conversion
(test_dataset)

Dataset({
    features: ['labels', 'post', '__index_level_0__'],
    num_rows: 5983
})

In [61]:
#remove unnecessary column
test_dataset=test_dataset.remove_columns("__index_level_0__")
train_dataset=train_dataset.remove_columns("__index_level_0__")
evaluation_dataset=evaluation_dataset.remove_columns("__index_level_0__")

In [62]:
#combine the train and test dataset into one datset
main_dataset= datasets.DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'evaluate': evaluation_dataset
})

In [63]:
#preview of the dataset after merging
main_dataset

DatasetDict({
    train: Dataset({
        features: ['labels', 'post'],
        num_rows: 24017
    })
    test: Dataset({
        features: ['labels', 'post'],
        num_rows: 5983
    })
    evaluate: Dataset({
        features: ['labels', 'post'],
        num_rows: 2400
    })
})

In [64]:
# training and testing data size
training_data_size = main_dataset['train'].num_rows
testing_data_size = main_dataset['test'].num_rows
evaluation_data_size = main_dataset['evaluate'].num_rows

In [65]:
#loading a tokenizer from the pretrained model
tokenizer = AutoTokenizer.from_pretrained("israel/AmharicRoBERTa")

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--israel--AmharicRoBERTa/snapshots/00692edb3619168e324cd49db01f2789ab49cf4a/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--israel--AmharicRoBERTa/snapshots/00692edb3619168e324cd49db01f2789ab49cf4a/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--israel--AmharicRoBERTa/snapshots/00692edb3619168e324cd49db01f2789ab49cf4a/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--israel--AmharicRoBERTa/snapshots/00692edb3619168e324cd49db01f2789ab49cf4a/tokenizer_config.json


In [66]:
#Have a tokenizer function that uses the tokenizer 
def tokenize_function(data):
    return tokenizer(data["post"], padding="max_length", truncation=True)

In [67]:
#Tokenize all the data using the mapping functionality
tokenized_datasets = main_dataset.map(tokenize_function)


  0%|          | 0/24017 [00:00<?, ?ex/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/5983 [00:00<?, ?ex/s]

  0%|          | 0/2400 [00:00<?, ?ex/s]

In [68]:
#empty cache
torch.cuda.empty_cache()

In [69]:
#remove the posts column as it is no longer needed
tokenized_datasets = tokenized_datasets.remove_columns(["post"])

In [70]:
#changing the format of the tokenized dataset to torch
tokenized_datasets.set_format("torch")

In [71]:
#shuffeling and selecting the needed size of dataset for training and evaluating the model
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(training_data_size))
small_test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(testing_data_size))
small_eval_dataset = tokenized_datasets["evaluate"].shuffle(seed=42).select(range(evaluation_data_size))

In [72]:
# preview of the shuffeled and selected evaluation dataset
small_eval_dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2400
})

In [73]:
# preview of the shuffeled and selected training dataset
small_train_dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 24017
})

In [74]:
# preview of the shuffeled and selected testing dataset
small_test_dataset

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 5983
})

In [75]:
#load the dataset using DataLoader
train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=4)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=4)
test_dataloader = DataLoader(small_test_dataset, batch_size=4)

In [76]:
#Load auto mode classifier from the pretrained model 
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("israel/AmharicRoBERTa", num_labels=2)
# model = AutoModelForSequenceClassification.from_pretrained("Davlan/afro-xlmr-large", num_labels=2)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--israel--AmharicRoBERTa/snapshots/00692edb3619168e324cd49db01f2789ab49cf4a/config.json
Model config RobertaConfig {
  "_name_or_path": "israel/AmharicRoBERTa",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 30000
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--israel--AmharicRoBERTa/sna

In [77]:
# #load an optimizer
# optimizer = AdamW(model.parameters(), lr=5e-5)
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [99]:
import numpy as np
from datasets import load_metric

metric = load_metric("f1","accuracy")

In [95]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [80]:
from transformers import TrainingArguments, Trainer

In [81]:
from transformers import EarlyStoppingCallback, IntervalStrategy

In [41]:
# training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", num_train_epochs=2)

In [100]:
training_args = TrainingArguments(
   f"training_with_callbacks",
   evaluation_strategy = IntervalStrategy.STEPS, # "steps"
   warmup_steps=500,                # number of warmup steps for learning rate  
   save_steps=2000,
   eval_steps = 2000, # Evaluation and Save happens every 50 steps
   save_total_limit = 3, # Only last 5 models are saved. Older ones are deleted.
   learning_rate=1e-5,
   per_device_train_batch_size=1,
   per_device_eval_batch_size=1,
   num_train_epochs=5,
   weight_decay=0.01,
   push_to_hub=False,
   metric_for_best_model = 'f1',
   load_best_model_at_end=True)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [101]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=10)],
)

In [84]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 24017
  Num Epochs = 5
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 120085
  Number of trainable parameters = 66556418


Step,Training Loss,Validation Loss,F1
2000,1.380200,1.263784,0.719481
4000,1.340500,1.356597,0.642526
6000,1.283800,1.338232,0.715676
8000,1.361000,1.469770,0.670501
10000,1.511100,1.217923,0.679182
12000,1.329500,1.296401,0.693257
14000,1.377500,1.277777,0.692082
16000,1.390200,1.232271,0.740824
18000,1.347500,1.302099,0.667261
20000,1.293700,1.164994,0.744689


***** Running Evaluation *****
  Num examples = 2400
  Batch size = 1
Saving model checkpoint to training_with_callbacks/checkpoint-2000
Configuration saved in training_with_callbacks/checkpoint-2000/config.json
Model weights saved in training_with_callbacks/checkpoint-2000/pytorch_model.bin
Deleting older checkpoint [training_with_callbacks/checkpoint-4000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 2400
  Batch size = 1
Saving model checkpoint to training_with_callbacks/checkpoint-4000
Configuration saved in training_with_callbacks/checkpoint-4000/config.json
Model weights saved in training_with_callbacks/checkpoint-4000/pytorch_model.bin
Deleting older checkpoint [training_with_callbacks/checkpoint-6000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 2400
  Batch size = 1
Saving model checkpoint to training_with_callbacks/checkpoint-6000
Configuration saved in training_with_callbacks/checkpoint-6000/config.json
Model we

TrainOutput(global_step=120085, training_loss=0.9068586176756186, metrics={'train_runtime': 6405.9982, 'train_samples_per_second': 18.746, 'train_steps_per_second': 18.746, 'total_flos': 927539571368580.0, 'train_loss': 0.9068586176756186, 'epoch': 5.0})

In [102]:
trainer.evaluate(small_test_dataset)

***** Running Evaluation *****
  Num examples = 2400
  Batch size = 1


{'eval_loss': 0.6575053334236145,
 'eval_f1': 0.9065800162469536,
 'eval_runtime': 22.6922,
 'eval_samples_per_second': 105.763,
 'eval_steps_per_second': 105.763}

In [87]:
#install huggingface_hub package to interact with huggingface platform
!pip install huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [89]:
from huggingface_hub import notebook_login

In [90]:
#login to huggingface
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [91]:
#specify the path for the trainde model and tokenizer to huggingface repository
model.save_pretrained("path/to/amharic-hate-speech-detection-finetuned-Roberta")
tokenizer.save_pretrained("path/to/amharic-hate-speech-detection-finetuned-Roberta")

Configuration saved in path/to/amharic-hate-speech-detection-finetuned-Roberta/config.json
Model weights saved in path/to/amharic-hate-speech-detection-finetuned-Roberta/pytorch_model.bin
tokenizer config file saved in path/to/amharic-hate-speech-detection-finetuned-Roberta/tokenizer_config.json
Special tokens file saved in path/to/amharic-hate-speech-detection-finetuned-Roberta/special_tokens_map.json


('path/to/amharic-hate-speech-detection-finetuned-Roberta/tokenizer_config.json',
 'path/to/amharic-hate-speech-detection-finetuned-Roberta/special_tokens_map.json',
 'path/to/amharic-hate-speech-detection-finetuned-Roberta/vocab.txt',
 'path/to/amharic-hate-speech-detection-finetuned-Roberta/added_tokens.json',
 'path/to/amharic-hate-speech-detection-finetuned-Roberta/tokenizer.json')

In [92]:
#push the trained model to huggingface repository
model.push_to_hub("amharic-hate-speech-detection-finetuned-Roberta")

Configuration saved in /tmp/tmpjsdr_p6z/config.json
Model weights saved in /tmp/tmpjsdr_p6z/pytorch_model.bin
Uploading the following files to amengemeda/amharic-hate-speech-detection-finetuned-Roberta: config.json,pytorch_model.bin


CommitInfo(commit_url='https://huggingface.co/amengemeda/amharic-hate-speech-detection-finetuned-Roberta/commit/7117a6cdd8cce48d9a5c68af67d8f4c15c2ef961', commit_message='Upload RobertaForSequenceClassification', commit_description='', oid='7117a6cdd8cce48d9a5c68af67d8f4c15c2ef961', pr_url=None, pr_revision=None, pr_num=None)

In [93]:
#push the tokenizer to huggingface repository
tokenizer.push_to_hub("amharic-hate-speech-detection-finetuned-Roberta")

tokenizer config file saved in /tmp/tmp2i6digfm/tokenizer_config.json
Special tokens file saved in /tmp/tmp2i6digfm/special_tokens_map.json
Uploading the following files to amengemeda/amharic-hate-speech-detection-finetuned-Roberta: tokenizer.json,vocab.txt,tokenizer_config.json,special_tokens_map.json


CommitInfo(commit_url='https://huggingface.co/amengemeda/amharic-hate-speech-detection-finetuned-Roberta/commit/828147d99e7ee75fe4d3fe90c0716db87737482d', commit_message='Upload tokenizer', commit_description='', oid='828147d99e7ee75fe4d3fe90c0716db87737482d', pr_url=None, pr_revision=None, pr_num=None)

In [49]:
model.save_pretrained("/content/drive/MyDrive/Machine Learning/fine-tuned_BERT/model_5")

Configuration saved in /content/drive/MyDrive/Machine Learning/fine-tuned_BERT/model_5/config.json
Model weights saved in /content/drive/MyDrive/Machine Learning/fine-tuned_BERT/model_5/pytorch_model.bin
